In [ ]:
import os
import torch
# import cv2
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from kornia.feature import DenseSIFTDescriptor, SIFTDescriptor
from kmeans_pytorch import kmeans
from sklearn.cluster import KMeans

In [ ]:
print(torch.cuda.is_available())
device = torch.device('cuda:0')
print(torch.cuda.get_device_name())

In [ ]:
code_folder = os.getcwd()
annotation_folder = os.path.join(code_folder,'../../dataset/Stanford/Annotation')
images_folder = os.path.join(code_folder,'../../dataset/Stanford/Images')

In [ ]:
transform = transforms.Compose([transforms.Resize((255,255)), transforms.Grayscale(num_output_channels=1), transforms.ToTensor()])
dataset = datasets.ImageFolder(images_folder, transform=transform)

In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [ ]:
descs = None
labels_all = None
for images, labels in dataloader:
    SIFT = SIFTDescriptor(patch_size=255)
    desc = SIFT(images)
    desc = desc.to(device=device)
    labels = labels.to(device=device)
    if descs is None:
        descs = desc
        labels_all = labels
    else:
        descs = torch.cat((descs, desc))
        labels_all = torch.cat((labels_all, labels))
print(descs.shape)

In [ ]:
for cluster in range(10):
    cluster_ids_x, cluster_centers = kmeans(X=descs, num_clusters=3, distance='euclidean', device=device)
    cluster_ids_x = cluster_ids_x.to(device=device)
    diff = torch.abs(cluster_ids_x - labels_all)
    diff[diff == 2] = 1
    error_rate = diff.sum() / diff.shape[0]
    accuracy = 1 - error_rate
    print(cluster)
    print(accuracy)

In [ ]:
descs_numpy = descs.cpu().detach().numpy()
kmeans = KMeans(n_clusters=3).fit(descs_numpy)
results_torch = torch.from_numpy(kmeans.labels_).to(device)
diff = torch.abs(results_torch - labels_all)
diff[diff == 2] = 1
error_rate = diff.sum() / diff.shape[0]
accuracy = 1 - error_rate
print(accuracy)